In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchviz import make_dot
import hiddenlayer as hl

In [2]:
class Conv1DNet(nn.Module):
    def __init__(self, 
                 input_channels=1,    # Number of input channels (1 for a single "row" input)
                 hidden_channels=32,  # Number of hidden channels
                 kernel_size=3):
        super(Conv1DNet, self).__init__()
        
        self.conv1 = nn.Conv1d(in_channels=input_channels,
                               out_channels=hidden_channels,
                               kernel_size=kernel_size,
                               padding=(kernel_size // 2))
        self.conv2 = nn.Conv1d(in_channels=hidden_channels,
                               out_channels=hidden_channels,
                               kernel_size=kernel_size,
                               padding=(kernel_size // 2))
        # Final layer outputs back to 1 channel
        self.conv3 = nn.Conv1d(in_channels=hidden_channels,
                               out_channels=input_channels,
                               kernel_size=kernel_size,
                               padding=(kernel_size // 2))

    def forward(self, x):
        """
        x shape: (batch_size, 1, N)
        """
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        # No activation on final layer if you want raw output of size (batch_size, 1, N)
        x = self.conv3(x)  
        return x

In [3]:
model = Conv1DNet()
dummy_input = torch.randn(1, 1, 100)

# Export to ONNX
torch.onnx.export(
    model, 
    dummy_input, 
    "conv1d_model.onnx",
    input_names=["input"],
    output_names=["output"],
    opset_version=11
)